In [ ]:
import pandas as pd
import numpy as np
from astropy.cosmology import Planck18 as cosmo
import matplotlib.pyplot as plt
from GWFish.modules.detection import Network
from GWFish.modules.fishermatrix import compute_network_errors

# IMBH population

In [ ]:
rng = np.random.default_rng(seed=1)
ns = 10
total_masses = rng.normal(loc=1e4, scale=1e3, size=(ns,))
mass_ratios = rng.uniform(1, 10, size=(ns,))
redshifts = rng.uniform(0.5, 10, size=(ns,))

parameters = pd.DataFrame.from_dict({
   'mass_1': total_masses / (1+mass_ratios), 
   'mass_2': total_masses * mass_ratios / (1+mass_ratios), 
   'redshift': redshifts,
   'luminosity_distance': cosmo.luminosity_distance(redshifts),
   'theta_jn': np.arccos(rng.uniform(-1., 1., size=(ns,))),
   'dec': np.arccos(rng.uniform(-1., 1., size=(ns,))) - np.pi / 2.,
   'ra': rng.uniform(0, 2. * np.pi, size=(ns,)),
   'psi': rng.uniform(0, 2. * np.pi, size=(ns,)),
   'phase': rng.uniform(0, 2. * np.pi, size=(ns,)),
   'geocent_time': rng.uniform(1735257618, 1766793618, size=(ns,)) # full year 2035
})

In [ ]:
# Plot of primary against secondary masses
plt.scatter(parameters['mass_1'], parameters['mass_2'])
plt.xlabel('Primary mass [$M_\odot$]')
plt.ylabel('Secondary mass [$M_\odot$]')

In [ ]:
network = Network(['LGWA'], detection_SNR=(0, 0))
detected, snr, errors, _ = compute_network_errors(
   network, 
   parameters, 
   waveform_model='IMRPhenomXPHM',
   fisher_parameters=['geocent_time'],
)

snr = snr[detected]
errors = errors[detected, :]
snr